In [7]:
import pandas as pd
df = pd.read_csv("spam.csv",encoding = "ISO-8859-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 'columns')
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df = df.rename(columns = {'v1':'label','v2':'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [11]:
ham = df[df['label']=='ham']
ham.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...


In [12]:
spam = df[df['label']=='spam']
spam.head()

,label,text
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [13]:
ham.shape,spam.shape

((4825, 2), (747, 2))

In [14]:
ham = ham.sample(spam.shape[0])
ham.shape,spam.shape

((747, 2), (747, 2))

In [15]:
data = ham.append(spam,ignore_index = True)
data.tail()

,label,text
1489,spam,Want explicit SEX in 30 secs? Ring 02073162414...
1490,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
1491,spam,Had your contract mobile 11 Mnths? Latest Moto...
1492,spam,REMINDER FROM O2: To get 2.50 pounds free call...
1493,spam,This is the 2nd time we have tried 2 contact u...


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [17]:
X_train,X_test,y_train,y_test = train_test_split(data['text'],data['label'],test_size = 0.2,shuffle = True)

In [18]:
X_train.head()

450    They have a thread on the wishlist section of ...
341    Gal n boy walking in d park. gal-can i hold ur...
123                            Nah dub but je still buff
633    Yup. Izzit still raining heavily cos i'm in e ...
972    As a Registered Subscriber yr draw 4 a å£100 g...
Name: text, dtype: object

In [19]:
### Bag of words

vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

In [29]:
X_test_vect = vectorizer.transform(X_test)

In [24]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [25]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params':{
            'C':[10,100,1000],
            'kernel':['rbf','linear']
        }
    },
    'random_forest':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators':[5,10,20],
            'criterion':['gini','entropy']
        }
    }
}

In [34]:
from sklearn.model_selection import RandomizedSearchCV

scores = []

for model_name, mp in model_params.items():
    clf_rcv = RandomizedSearchCV(mp['model'],mp['params'],cv=5,n_iter =2,return_train_score=False)
    clf_rcv.fit(X_train_vect,y_train)
    scores.append({
        'model':model_name,
        'best_score':clf_rcv.best_score_,
        'best_params':clf_rcv.best_params_
    })

df_result = pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [35]:
df_result

,model,best_score,best_params
0,svm,0.957322,"{'kernel': 'linear', 'C': 1000}"
1,random_forest,0.942259,"{'n_estimators': 20, 'criterion': 'gini'}"


### the best model for this dataset is svm with perticular parameters 

In [36]:
y_pred = clf_rcv.predict(X_test_vect)

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[145,   2],
       [ 12, 140]])